In [1]:
import h5py
import numpy as np
np.random.seed(1337)
from keras.optimizers import SGD
import logging

# In[2]:

h5f = h5py.File('preprocessed_data_winSize100_winShift10.h5','r')
training_data = h5f['training_data'][:]
training_output = h5f['training_output'][:]
testing_data = h5f['testing_data'][:]
testing_output = h5f['testing_output'][:]

h5f = h5py.File('preprocessed_data_winSize100_winShift10_mean_sd.h5','r')
mean = h5f['mean'][:]
sd = h5f['sd'][:]
h5f.close()

print(training_data.shape,training_output.shape,testing_data.shape,testing_output.shape)


Using TensorFlow backend.


((218288, 1200), (218288,), (93552, 1200), (93552,))


In [2]:
print("Concatenating training")
training = np.concatenate([training_data, training_output.reshape(training_output.shape[0],1)], axis=1)
print("Deleting training")
del training_data
del training_output
print("Shuffling training")
np.random.shuffle(training)

print("Concatenating testing")
testing = np.concatenate([testing_data, testing_output.reshape(testing_output.shape[0],1)], axis=1)
print("Deleting testing")
del testing_data
del testing_output
print("Shuffling Testing")
np.random.shuffle(testing)

print("Getting back train data and output")
training_data=training[:,:-1]
training_output=training[:,-1]
print("Deleting training")
del training

print("Getting back test data and output")
testing_data=testing[:,:-1]
testing_output=testing[:,-1]
print("Deleting testing")
del testing

print(training_data.shape,training_output.shape,testing_data.shape,testing_output.shape)


Concatenating training
Deleting training
Shuffling training
Concatenating testing
Deleting testing
Shuffling Testing
Getting back train data and output
Deleting training
Getting back test data and output
Deleting testing
((218288, 1200), (218288,), (93552, 1200), (93552,))


In [3]:

testing_data=testing_data-mean
training_data=training_data-sd

print(training_data.shape,training_output.shape)


((218288, 1200), (218288,))


In [4]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils


# In[11]:
nb_classes = 7

training_output = np_utils.to_categorical(training_output, nb_classes)
testing_output = np_utils.to_categorical(testing_output, nb_classes)
print(training_output.shape)


/usr/local/lib/python2.7/dist-packages/keras/utils/np_utils.py:23: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Y[i, y[i]] = 1.


(218288, 7)


In [5]:
from keras.regularizers import l2

batch_size = 512

nb_epoch = 10
nb_neg_cycles = 3

print(training_data.shape)

logging.basicConfig(filename= "./WSDM_Keras_batch_size"+"("+str(batch_size)+")"+".log", level=logging.INFO, format='%(message)s')
logging.info("Parameters")

(218288, 1200)


In [13]:

model = Sequential()


model.add(Dense(256, input_shape=(training_data.shape[1],)))
model.add(Activation('relu'))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_8 (Dense)                  (None, 256)           307456      dense_input_3[0][0]              
____________________________________________________________________________________________________
activation_8 (Activation)        (None, 256)           0           dense_8[0][0]                    
____________________________________________________________________________________________________
dense_9 (Dense)                  (None, 7)             1799        activation_8[0][0]               
____________________________________________________________________________________________________
activation_9 (Activation)        (None, 7)             0           dense_9[0][0]                    
Total params: 309255
______________________________________________________________________

In [14]:

epoch = 1

final_acc=0.0
lr = 0.0001
weights=[]
first_run=True
flag=True
iter_count = 1
count_neg_iter=0


#The flag to keep the loop running
run_flag=True
weights=[]

#Check if it is the first iteration
first_iter=True


In [15]:
while run_flag:

    #Give ramdom weights in first iteration and previous weights to other
    if first_iter:
        first_iter=False
    else:
        model.set_weights(np.asarray(weights))


    logging.info("\n\nIteration:"+str(iter_count))
    print ("Learning Rate : ",lr)
    logging.info("Learning Rate: "+str(lr))
    
    sgd = SGD(lr=lr)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

    model.fit(training_data, training_output, batch_size=batch_size, nb_epoch=1,
              verbose=1, validation_data=(testing_data, testing_output))
    score = model.evaluate(testing_data, testing_output, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])
    
    current_acc = score[1]

    logging.info(current_acc)

    if(current_acc - final_acc > 0.0005):        
        iter_count = iter_count + 1

        #Update the weights if the accuracy is greater than .001
        weights=model.get_weights()
        print ("Updating the weights")
        logging.info("Updating the weights")
        #Updating the final accuracy
        final_acc=current_acc
        #Setting the count to 0 again so that the loop doesn't stop before reducing the learning rate n times consecutivly
        count_neg_iter = 0

    else:
        #If the difference is not greater than 0.005 reduce the learning rate
        lr=lr/2.0
        print ("Reducing the learning rate by half")
        logging.info("Reducing the learning rate by half")
        count_neg_iter = count_neg_iter + 1
        
        #If the learning rate is reduced consecutively for nb_neg_cycles times then the loop should stop
        if(count_neg_iter>nb_neg_cycles):
            run_flag=False
            model.set_weights(np.asarray(weights))

('Learning Rate : ', 0.0001)
Train on 218288 samples, validate on 93552 samples
Epoch 1/1
218288/218288 [==============================] - 7s - loss: 1.6717 - acc: 0.7800 - val_loss: 1.8902 - val_acc: 0.7590
('Test score:', 1.8902443540143175)
('Test accuracy:', 0.7590064805492247)
Updating the weights
('Learning Rate : ', 0.0001)
Train on 218288 samples, validate on 93552 samples
Epoch 1/1
218288/218288 [==============================] - 7s - loss: 1.2395 - acc: 0.8037 - val_loss: 1.7775 - val_acc: 0.7585
('Test score:', 1.7774774218412961)
('Test accuracy:', 0.7585437905386776)
Reducing the learning rate by half
('Learning Rate : ', 5e-05)
Train on 218288 samples, validate on 93552 samples
Epoch 1/1
218288/218288 [==============================] - 7s - loss: 1.2869 - acc: 0.8013 - val_loss: 1.8271 - val_acc: 0.7576
('Test score:', 1.8271492387120165)
('Test accuracy:', 0.75761840641957301)
Reducing the learning rate by half
('Learning Rate : ', 2.5e-05)
Train on 218288 samples, valid

In [12]:
print('Final accuracy:', final_acc)

logging.info(final_acc)

('Final accuracy:', 0.79569806586947833)
